In [9]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root="../../data",  # 데이터를 저장할 root 디렉토리
    train=True,  # 훈련용 데이터 설정
    download=True,  # 다운로드
    transform=ToTensor(),  # 이미지 변환. 여기서는 TorchTesnor로 변환시킵니다.
)

test_data = datasets.FashionMNIST(
    root="../../data", train=False, download=True, transform=ToTensor()
)

In [10]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [11]:
import torch

def select_device():
    if torch.backends.mps.is_available():
        device = "mps"
        print("MPS 가속을 사용합니다.")
    else:
        device = torch.device("cpu")
        print("MPS 가속을 사용할 수 없습니다. CPU를 사용합니다.")

# select_device()
device = torch.device("cpu")

In [12]:
from torch import nn


class NeuralNetwork(nn.Module):
    def __init__(self):
        # subclass 인 neural network 의 생성자
        # 상위 클래스인 nn.Module 의 생성에 대한 책임을 져야 한다. => super 를 통해 상위 클래스의 생성자 호출
        super(NeuralNetwork, self).__init__()

        # 생성자에는 항상 레이어의 구성을 정의
        self.flatten = nn.Flatten()  # 입력되는 3차원 데이터를 평탄화

        # nn.Sequential 을 이용해 연속되는 레이어의 구조를 구성
        self.linear_relu_stack = nn.Sequential(
            # 1층의 구조
            nn.Linear(28 * 28, 128),  # 28*28 을 받는 128개의 뉴런
            nn.ReLU(),
            # 2층(출력층) 구조
            nn.Linear(128, 10),
            # Softmax 는 나중에 실제 모델에서 확률값을 기반한 훈련 시 사용해도 됨
        )

    def forward(self, x):
        # forward 에는 gray scale 입력 데이터 x 가 들어옴 => x 의 shape 은 (N, 28, 28)
        x = self.flatten(x)
        y = self.linear_relu_stack(x)

        return y

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


# 모델 훈련

파이토치의 모델 훈련을 위해서는 손실함수, 최적화 함수를 등록해야 합니다. 특히 최적화 함수를 사용하기 위해서는 `model.parameters()` 메소드를 이용해 최적화 대상 파라미터를 지정해주면 됩니다.

In [14]:
loss_fn = nn.CrossEntropyLoss()

# optimizer : 경사하강법을 수행하기 위한 함수
# 경사하강법은 parameters(W, b) 에 수행됨
# 모델에서 파라미터를 꺼내 최적화 함수에 등록
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# 훈련 과정 (define training loop)
1. dataloader 에서 데이터를 꺼낸다
2. 데이터를 모델에 통과시킨다 (순전파를 통한 prediction)
3. 얻어낸 예측값을 이용해 loss 계산
4. 역전파를 통해 미분값 계산
5. 얻어낸 미분 값으로 경사하강법 수행 (optimization)

In [15]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # 모델을 훈련 모드로 설정
    model.train()

    # 데이터 꺼내기. for문을 사용하면 자동으로 next(iter(dataloader))가 실행됨
    for batch, (X, y) in enumerate(dataloader):
        # 현재 dataloader에 있는 데이터는 cpu에 존재 => gpu로 옮겨준다
        # 모델이 위치한 곳과 데이터가 위치한 곳을 동일하게 맞춰야 함
        X, y = X.to(device), y.to(device)

        # 순전파 수행
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)  # 자동으로 Softmax 적용

        # 역전파 수행
        optimizer.zero_grad()  # 기존에 남아있던 기울기를 제거 (이전 batch 의 기울기가 남아있으면 정확한 값 얻기 힘들다)
        loss.backward()  # 역전파, loss 가 leaf
        optimizer.step()  # 구한 미분값을 토대로 최적화 수행 (경사하강법)

        # 배치가 100번 돌 때마다 화면에 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

# 추론을 위한 test loop 정의

1. test dataloader 에서 데이터 꺼내기
2. 순전파를 통한 예측값 얻기
3. metric 계산 (loss, accuracy)
    * batch 별 평균 성능 계산

In [16]:
def test_loop(dataloader, model, loss_fn):  # test 에서는 optimizer 필요없다
    size = len(dataloader.dataset)

    test_loss, correct = 0, 0

    # 평가모드(추론모드) 설정
    model.eval()

    # 추론 과정은 기울기를 구할 필요가 없다
    with torch.no_grad():  # 기울기를 구하지 않는 모드로 전환
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # 맞힌 갯수 총합
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    num_batches = len(dataloader)

    test_loss /= num_batches
    correct /= size

    print(
        f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n"
    )

훈련 수행

In [17]:
epochs = 10

for i in range(epochs):
    print(f"Epochs {i+1}\n..................")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Experiment Successful")

Epochs 1
..................
Train Loss : 2.281793 [     0 / 60000 ]
Train Loss : 1.438720 [  6400 / 60000 ]
Train Loss : 1.018364 [ 12800 / 60000 ]
Train Loss : 0.903798 [ 19200 / 60000 ]
Train Loss : 0.763800 [ 25600 / 60000 ]
Train Loss : 0.819863 [ 32000 / 60000 ]
Train Loss : 0.668227 [ 38400 / 60000 ]
Train Loss : 0.657638 [ 44800 / 60000 ]
Train Loss : 0.708049 [ 51200 / 60000 ]
Train Loss : 0.621674 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 78.9%, Avg Loss : 0.628503

Epochs 2
..................
Train Loss : 0.501942 [     0 / 60000 ]
Train Loss : 0.592810 [  6400 / 60000 ]
Train Loss : 0.563680 [ 12800 / 60000 ]
Train Loss : 0.394578 [ 19200 / 60000 ]
Train Loss : 0.536491 [ 25600 / 60000 ]
Train Loss : 0.510127 [ 32000 / 60000 ]
Train Loss : 0.672792 [ 38400 / 60000 ]
Train Loss : 0.482456 [ 44800 / 60000 ]
Train Loss : 0.598024 [ 51200 / 60000 ]
Train Loss : 0.454794 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 81.4%, Avg Loss : 0.539282

Epochs 3
..................
Train Lo